# Bleaching samples

## update sample type 
- everything is healthy or diseased right now, 
-> change to bleached based sampling notes
-> then crosscheck with colonydata?

In [1]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
# upload sample and colony data 
sample=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_samples.csv')
colony=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv')

In [3]:
sample.head()

,Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,Species,Time_sampled,Time_processed,Sample_type,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes
0,122022,BEL,CBC,12/5/22,CURLEW,4,NaN,77,OFAV,NaN,NaN,Core_frozen,18,Healthy,NaN,122022_BEL_CBC_T4_18_OFAV,NaN,NaN,NaN,NaN
1,92023,BEL,CBC,9/27/23,CBC30N,1,NaN,1,SSID,NaN,NaN,Core_RNAlater,185,Diseased_Margin,only margin sample available,092023_BEL_CBC_T1_185_SSID,UML_NARWHAL_R1_B10,NaN,NaN,NaN
2,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,2,PAST,NaN,NaN,Core_RNAlater,171,Healthy,CLP 90%,092023_BEL_CBC_T1_171_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
3,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,3,SSID,NaN,NaN,Core_RNAlater,173,Healthy,CLP 80%; DC 20%,092023_BEL_CBC_T1_173_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
4,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,12,PSTR,NaN,NaN,Core_RNAlater,177,Healthy,No CL,092023_BEL_CBC_T1_177_PSTR,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN


In [4]:
colony.head()

,Unnamed: 0,Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,...,082024_Percentage,Notes_082024,122024_Condition,122024_Percentage,Notes_122024,062025_Condition,062025_Percentage,Notes_062025,immune_y/n,checked_colonies
0,1.0,6/21/19,CBC30N,1,349,1,SSID,1.5,0.5,right,...,NaN,NaN,Not_visited,NaN,Diseased,Diseased,NaN,NaN,n,x
1,2.0,6/21/19,CBC30N,1,334,2,PAST,1.4,0.1,left,...,NaN,sample lower colony (original colony); inc old...,NaN,NaN,NaN,Healthy,NaN,NaN,y,x
2,3.0,6/21/19,CBC30N,1,346,3,SSID,6.3,0.8,right,...,NaN,some discoloration,DC,10%,30% mortality,Diseased,NaN,NaN,y,x
3,4.0,10/15/19,CBC30N,1,407,4,PSTR,6.5,0.5,right,...,NaN,NaN,Dead,NaN,Dead,Dead,NaN,NaN,n,x
4,5.0,6/24/19,CBC30N,1,394,5,SSID,4.3,1.2,left,...,NaN,NaN,DC,10%,80% mortality,Diseased,NaN,NaN,n,x


In [5]:
# search for sampling notes with clp or clb 

#if col contains clb or clp 
# replace status with bleached tissue 
sample.loc[
    sample['Sampling_notes'].str.contains('CLP|CLB', case=False, na=False),'Health_status']= 'Bleached_Tissue'
# (code is in place replacement)

In [6]:
sample.loc[:,'Health_status']

0               Healthy
1       Diseased_Margin
2       Bleached_Tissue
3       Bleached_Tissue
4               Healthy
             ...       
2524            Healthy
2525            Healthy
2526            Healthy
2527            Healthy
2528            Healthy
Name: Health_status, Length: 2529, dtype: object

In [7]:
# match sample type to colony statuses
# got from my cbc_metagenomics code

In [8]:
# filter for UML samples
sample_subset=sample[
    (sample['Sample_type'] == 'Core_EtOH') |
    (sample['Sample_type'] == 'Core_RNAlater')
]

In [9]:
# add colony ID - t# newtagnum species
sample_subset = sample_subset.copy()
sample_subset['TransectNum_str'] = 'T' + sample_subset['TransectNum'].astype(str)
sample_subset['colony_id'] = sample_subset[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
sample_subset.drop(columns='TransectNum_str', inplace=True)

In [10]:
# add colony ID - t# newtagnum species
colony = colony.copy()
colony['TransectNum_str'] = 'T' + colony['TransectNum'].astype(str)
colony['colony_id'] = colony[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
colony.drop(columns='TransectNum_str', inplace=True)

In [ ]:
## BELOW SCRIPT IS IN PROGRESS 8/20/2025 

In [134]:
# create function to crosscheck colony health with samples 

def status_match(group):
    # get the condition and list sample statuses of each colony at each monthyear (by row) 
    cond = str(group['colony_condition'].iloc[0])
    # turned into string to prevent crashes from NAs
    statuses = group['sample_condition'].dropna().tolist()
    month = group['Month_year'].iloc[0]
    immune_dates = [112023, 122023, 12024, 22024, 62024, 82024, 122024] 

    # match colony conditions to sample conditions 
    # if colony_condition is healthy, sample = healthy 
    if cond == 'Healthy':
        return all(s == 'Healthy' for s in statuses)

    # if condition is diseased healthy = diseased_margin and diseased_tissue (but rn these are on 2 diff rows)
    elif cond in ['Diseased','Diseased_Other']:
        return all(x in statuses for x in ['Diseased_Tissue', 'Diseased_Margin'])

    # if condition is clp | clb = bleached_tissue 
    elif any(term in cond for term in ['CLP','CLB']):
        return all(x in statuses for x in ['Bleached_Tissue'])
    
    # if condition is Not_visited, sample = NaN
    # if condition is Dead, sample = NaN 
    elif cond in ['Dead', 'Not_Visited','Not_visited','not_visited','NAN']:
        return all(pd.isna(s) for s in group['sample_condition'])

    # if condition is na, sample should also be na 
    elif pd.isna(cond):
        return all(pd.isna(s) for s in group['sample_condition'])

    # exclude False matches if the colony is non-immune during an immune sampling point 
    # list of immune dates 
    
    # if coral is NOT immune during an immune date 
    elif group['immune_y/n'].iloc[0]=='n' and month in immune_dates:
        # it is ok if the sample status is na 
        return all(pd.isna(s) for s in group['sample_condition'])

    ## UGH but this has to exclude T5 and T6 during 01/2024 and 042024 respectively because they were newly tagged then 

    # any mismatches return as false 
    else:
        return False

In [135]:
# create species list 
species_list=sample_subset['Species'].unique().tolist()
species_list

# ugh so many syntax errors...need to fix in 'sample_fixes'

['SSID',
 'PAST',
 'PSTR',
 'MCAV',
 'OANN',
 'DLAB',
 'OFAV',
 'CNAT',
 'DL',
 'MMEA',
 'PAST ',
 'Unknown',
 'OFAV ']

In [136]:
colony.columns

Index(['Unnamed: 0', 'Date_InitialTag', 'Transect', 'TransectNum', 'OldTagNum',
       'NewTagNum', 'Species', 'Meter', 'Meters_90', 'Direction', 'Size_Class',
       'MaxDiameter', 'Height', 'Date_DocumentedDisease',
       'Date_DocumentedMortality', 'Notes_062019', '062019_Condition',
       '062019_Percentage', 'Notes_052022', '052022_Condition',
       '052022_Percentage', 'Notes_122022', '122022_Condition',
       '122022_Percentage', 'Notes_092023', '092023_Condition',
       '092023_Percentage', 'Notes_112023', '112023_Condition',
       '112023_Percentage', 'Notes_122023', '122023_Condition',
       '122023_Percentage', 'Notes_012024', '012024_Condition',
       '012024_Percentage', 'Notes_022024', '022024_Condition',
       '022024_Percentage', 'Notes_042024', '042024_Condition',
       '042024_Percentage', '062024_Condition', '062024_Percentage',
       'Notes_062024', '082024_Condition', '082024_Percentage', 'Notes_082024',
       '122024_Condition', '122024_Percentage', 'N

In [137]:
# extract condition columns 
cond_cols=['colony_id'] + colony.columns[colony.columns.str.contains('_Condition')].tolist()

In [138]:
# create for loop for each specie - using the fxn to match the colony and sample health statuses 
species_checks = {}

for specie in species_list: 
    # filter colony and sample data for each specie 
    filtered_colony=colony[colony['Species']==specie]

    # extract condition columns 
    small_colony= filtered_colony.loc[:,cond_cols]
    # and pivot
    conditions_long=small_colony.melt(id_vars='colony_id', 
                              var_name='Month_year', 
                              value_name='colony_condition')
    
    # extract month_year from column names to match sample data 
    conditions_long['Month_year'] = conditions_long['Month_year'].str.extract(r'(\d+)').astype(int)
   
    # repeat for sample data 
    filtered_samples=sample_subset[sample_subset['Species']==specie]
    filtered_samples=filtered_samples.loc[:,('colony_id','Month_year','Health_status','Sampling_notes','Tubelabel_species')] 
    # rename column for sample health statuses 
    filtered_samples['sample_condition']=filtered_samples['Health_status']
    
    # merge dfs by colony id and monthyear
    merged = pd.merge(
    conditions_long.merge(filtered_colony, on='colony_id', how='left'),
    filtered_samples,
    on=['colony_id', 'Month_year'],
    how='outer'
    )

    checks = []
        # check each 'group' (unique combos of colony and monthyear) and store results in 'checks' list
    for (colony_id, month), group in merged.groupby(['colony_id', 'Month_year']):
        checks.append({
            'colony_id': colony_id,
            'immune_y/n': group['immune_y/n'].iloc[0],
            'Date_InitialTag': group['Date_InitialTag'].iloc[0],
            'Month_year': month,
                
            # run function &
            # create col containing results of the function that matches statuses
            'Match': status_match(group),
                
            'colony_condition': group['colony_condition'].iloc[0],
            'sample_statuses': group['sample_condition'].tolist(),
            'sample_ids': group['Tubelabel_species'].tolist(),
            'mortality_date' : group['Date_DocumentedMortality'].iloc[0]
    })
    # convert to dataframe and store each species df in a dictionary 'species_checks'
    species_checks[specie]=pd.DataFrame(checks)
    

In [139]:
species_checks['MCAV']['Month_year'].dtype

dtype('int64')

In [140]:
# check mismatches 
false_matches = {}
for specie in species_list:
    # make individual dfs per sp 
    df = pd.DataFrame(species_checks[specie])

    # exclude 122024 since no samples were taken 
    df = df[df['Month_year'] != 122024]
    
    # filter for where the match is false or if the colony condition is na and create a new df 
    false = df[(df['Match'] == False) & (~df['colony_condition'].isna())]
    
    # store false matches in new dictionary with species name as key 
    false_matches[specie] = false

In [141]:
species_list

['SSID',
 'PAST',
 'PSTR',
 'MCAV',
 'OANN',
 'DLAB',
 'OFAV',
 'CNAT',
 'DL',
 'MMEA',
 'PAST ',
 'Unknown',
 'OFAV ']

In [142]:
# view mismatches for each species 
false_matches['MCAV']

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
99,T1_7_MCAV,n,6/24/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T1_181_MCAV],12/1/24
114,T1_8_MCAV,n,6/24/19,122022,False,Diseased,[Diseased_Tissue],[122022_BEL_CBC_T1_144_MCAV],9/25/23
119,T2_53_MCAV,n,6/22/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1019_MCAV],Healthy
130,T2_55_MCAV,y,6/25/19,12024,False,CLP,[Healthy],[012024_BEL_CBC_T2_603_MCAV],Healthy
132,T2_55_MCAV,y,6/25/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1016_MCAV],Healthy
159,T2_59_MCAV,n,6/21/19,52022,False,Diseased,[nan],[nan],Diseased
164,T2_59_MCAV,n,6/21/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T2_220_MCAV],Diseased
180,T2_60_MCAV,y,6/22/19,122023,False,CLB,[Healthy],[122023_BEL_CBC_T2_522_MCAV],Healthy
203,T2_69_MCAV,y,6/25/19,92023,False,Healthy,[Bleached_Tissue],[092023_BEL_CBC_T2_176_MCAV],Healthy
227,T3_14_MCAV,y,6/23/19,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T3_1711_MCAV],Healthy


In [143]:
false_matches['PAST']

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
9,T1_13_PAST,y,6/24/19,112023,False,CLP,[nan],[nan],12/16/23
13,T1_19_PAST,n,6/21/19,12024,False,CLB,[nan],[nan],4/1/24
16,T1_19_PAST,n,6/21/19,52022,False,Diseased,[Diseased_Tissue],[052022_BEL_CBC_T1_52_PAST],4/1/24
21,T1_19_PAST,n,6/21/19,92023,False,CLB,[nan],[nan],4/1/24
26,T1_20_PAST,n,6/24/19,12024,False,CLP,[nan],[nan],4/26/24
39,T1_21_PAST,y,6/24/19,12024,False,CLP,[Healthy],[012024_BEL_CBC_T1_565_PAST],8/1/24
47,T1_21_PAST,y,6/24/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T1_172_PAST],8/1/24
52,T1_2_PAST,y,6/21/19,12024,False,"CLP,CLB",[nan],[nan],Healthy
121,T2_63_PAST,y,6/25/19,22024,False,Healthy,[Bleached_Tissue],[022024_Belize_CBC_T2_811_PAST],Healthy
157,T3_10_PAST,y,6/23/19,122023,False,CLB,[nan],[nan],Healthy


In [144]:
false_matches['SSID']
# why do we have some that say not visited but we have samples for..

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
13,T1_11_SSID,n,6/24/19,12024,False,Diseased,[nan],[nan],Diseased
15,T1_11_SSID,n,6/24/19,42024,False,Diseased,[Diseased_Margin],[042024_BEL_CBC_T1_936_SSID],Diseased
21,T1_11_SSID,n,6/24/19,92023,False,Diseased,[Diseased_Margin],[092023_BEL_CBC_T1_193_SSID],Diseased
26,T1_16_SSID,n,6/24/19,12024,False,Diseased,[nan],[nan],Diseased
32,T1_16_SSID,n,6/24/19,62025,False,Diseased,[nan],[nan],Diseased
...,...,...,...,...,...,...,...,...,...
323,T3_33_SSID,y,6/23/19,122023,False,DC,[Healthy],[122023_BEL_CBC_T3_534_SSID],Diseased
333,T3_36_SSID,n,6/23/19,92023,False,DC,[Healthy],[092023_BEL_CBC_T3_190_SSID],Diseased
338,T3_3_SSID,n,6/23/19,12024,False,Diseased,[nan],[nan],Diseased
366,T3_65_SSID,n,6/23/19,42024,False,Not_Visited,[Diseased_Margin],[042024_BEL_CBC_T3_964_SSID],Diseased


In [145]:
false_matches['PSTR']

,colony_id,immune_y/n,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
152,T2_54_PSTR,n,10/13/19,12024,False,CLP,[nan],[nan],Healthy
320,T3_70_PSTR,y,12/3/22,12024,False,CLP,[Healthy],[012024_BEL_CBC_T3_644_PSTR],Healthy
326,T3_70_PSTR,y,12/3/22,62025,False,CLP,[CLP],[062025_BEL_CBC_T3_1685_PSTR],Healthy
346,T3_75_PSTR,n,12/3/22,12024,False,CLP,[nan],[nan],Healthy
348,T3_75_PSTR,n,12/3/22,42024,False,Healthy,"[Diseased_Tissue, Diseased_Margin]","[042024_BEL_CBC_T3_965_PSTR, 042024_BEL_CBC_T3...",Healthy
354,T3_75_PSTR,n,12/3/22,92023,False,CLP,[Healthy],[092023_BEL_CBC_T3_192_PSTR],Healthy
367,T4_79_PSTR,y,12/5/22,92023,False,Healthy,[Bleached_Tissue],[092023_BEL_CBC_T4_93_PSTR],Healthy
411,T4_98_PSTR,y,12/5/22,12024,False,CLB,[Healthy],[012024_BEL_CBC_T4_661_PSTR],Healthy
413,T4_98_PSTR,y,12/5/22,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T4_1046_PSTR],Healthy
417,T4_98_PSTR,y,12/5/22,62025,False,Healthy,[Healthy],[062025_BEL_CBC_T4_1725_PSTR],Healthy


In [26]:
# if the colony condition says clb or clp, make the sample status bleached_tissue 
#make list of colony ids that have clb or clp in colony condition and healthy as sample status 

# false_matches
# for specie in species_list:
#     # filter dict for colony ids that have clb or clp in colony condition and healthy as sample status 
#     false_matches[specie]

{'SSID':       colony_id Date_InitialTag  Month_year  Match colony_condition  \
 13   T1_11_SSID         6/24/19       12024  False         Diseased   
 15   T1_11_SSID         6/24/19       42024  False         Diseased   
 21   T1_11_SSID         6/24/19       92023  False         Diseased   
 26   T1_16_SSID         6/24/19       12024  False         Diseased   
 32   T1_16_SSID         6/24/19       62025  False         Diseased   
 ..          ...             ...         ...    ...              ...   
 366  T3_65_SSID         6/23/19       42024  False      Not_Visited   
 372  T3_65_SSID         6/23/19       92023  False      Not_Visited   
 376  T3_65_SSID         6/23/19      122024  False              CLP   
 467  T6_14_SSID         4/29/24      122024  False         Healthy    
 480  T6_15_SSID         4/29/24      122024  False              CLP   
 
                         sample_statuses  \
 13                                [nan]   
 15                    [Diseased_Margi